In [1]:
import pandas as pd
import numpy as np
import altair as alt
import requests

'<!doctype html><html lang="en-US" dir="ltr"><head><base href="https://accounts.google.com/v3/signin/"><meta name="referrer" content="origin"><meta name="viewport" content="width=device-width, initial-scale=1"><style data-href="https://www.gstatic.com/_/mss/boq-identity/_/ss/k=boq-identity.AccountsSignInUi.I9HyC0AWXF4.L.W.O/am=MwKgAAQAECAAAAAAAAAAAAIACRE/d=1/ed=1/rs=AOaEmlFisTORSUK4NLP3t1lK3ynpDr6PKg/m=identifierview,_b,_tp,_r" nonce="9QxwrhTPhlrarh56cSn1_Q">c-wiz{contain:style}c-wiz>c-data{display:none}c-wiz.rETSD{contain:none}c-wiz.Ubi8Z{contain:layout style}.BrpTO{margin:auto;max-width:380px;overflow:hidden;position:relative}.BrpTO .Q8ElWe{position:relative;text-align:center}.viAgtf{-webkit-border-radius:50%;border-radius:50%;color:#5f6368;overflow:hidden}.eCirAf{line-height:1.4286}.cABCAe{width:100%}.cABCAe .viAgtf{-webkit-box-flex:none;-webkit-flex:none;flex:none;height:28px;margin-right:12px;width:28px}.cABCAe .Q8ElWe,.TPmpLe .Q8ElWe{display:-webkit-box;display:-webkit-flex;displ

In [ ]:
# The ID of spreadsheet.
database= '1ScmJ4rTC9DmHQjOdP-KiAD4cuP2UKfYrbhpdiEot82M' #url of google sheets truncated
service = build('sheets', 'v4')

# Call the Sheets API
sheet = service.spreadsheets()


revenue = sheet.values().get(spreadsheetId=database,
                            range="Revenue growth!A1:D32").execute()
values = revenue.get('values', [])

In [ ]:
df= pd.DataFrame(values)
df.head()

In [ ]:
df.columns = df.iloc[0]
df = df.drop(0)
df.columns = df.columns.str.lower()

In [ ]:
df.customs = df.customs.astype(float)
df.bir = df.bir.astype(float)
df.total = df.total.astype(float)

In [ ]:
revenues = alt.Chart(df).transform_fold(
    ['total', 'bir', 'customs']
).mark_line().encode(
    x='year:O',
    y='value:Q', 
    tooltip='value:Q',
    color='key:N'
).properties(width=700)

revenues

In [ ]:
gdp_ratio = sheet.values().get(spreadsheetId=database,
                            range="Revenue % of GDP!A1:H37").execute()
gdp_ratio = gdp_ratio.get('values', [])

In [ ]:
df2= pd.DataFrame(gdp_ratio)
df2.head()

In [ ]:
df2.columns = df2.iloc[0]
df2 = df2.drop(0)
df2.columns = df2.columns.str.lower()
df2.columns = df2.columns.str.replace(' ', "_", regex=False)
df2.head()

In [ ]:
df2.revenue = df2.revenue.astype(float)
df2.tax = df2.tax.astype(float)
df2.expenditures = df2.expenditures.astype(float)
df2.gdp_growth = df2.gdp_growth.astype(float)

In [ ]:
growth = alt.Chart(df2).mark_bar().encode(
    x='year:O',
    y="gdp_growth:Q",
    tooltip='gdp_growth:Q',
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.year == '2020',  # If the year is 2020 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('steelblue')   # And if it's not true it sets the bar steelblue.
    )
).properties(width=700)

growth

In [ ]:
deficit = alt.Chart(df2).mark_bar().encode(
    x='year:O',
    y="deficit:Q",
    tooltip='deficit:Q',
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.year == '2020',  # If the year is 2020 this test returns True,
        alt.value('red'),     # which sets the bar orange.
        alt.value('grey')   # And if it's not true it sets the bar steelblue.
    )
).properties(width=700)

deficit

In [ ]:
revenue.save('/charts/revenue.png')
deficit.save('/charts/deficit.png')
growth.save('/charts/growth.png')